In [7]:
!pip install fasttext
!git clone https://github.com/elsonidoq/ml-practico-2022.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3156315 sha256=16d5659516b3a3f0eb0b2051c6b19f98e26dc145439e59183d81692883693a5b
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
fatal: destination path 'ml-practico-2022' already exists and is not an empty directory.


In [1]:
import sys
from google.colab import drive

sys.path.append('ml-practico-2022/lib')
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from tqdm import tqdm
import re

# esta copiado de la notebook de word2vec para que sea comparable el resultado
word_pat = re.compile('\w+') # alfanumericos
num_pat = re.compile('\d+.\d*')

def tokenize(sent):
  return [
      tok for tok in word_pat.findall(sent.lower())
      if len(tok) > 1 and num_pat.match(tok) is None
  ]

def normalize(sent):
  return ' '.join(tokenize(sent))



class SentenceIterator:
  # esta clase es un "iterador reseteable" de oraciones para word2vec
  # que sea reseteable es que podes hacer `for thing in iterator` 
  # todas las veces que quieras
  def __init__(self, data):
    self.data = data

  def __iter__(self):
    for row in tqdm(self.data):
      yield normalize(row['title'])
      for line in row['description'].replace('. ', '\n').replace('<br/>', '\n').replace('<br>', '\n').split('\n'):
        sent = normalize(line)
        if sent: yield sent

In [3]:
import os
import csv
from taller_model_selection.evaluate import load_train_dev_test

PATH = '/content/gdrive/MyDrive/taller-model-selection-data/'
X_train, y_train = load_train_dev_test(PATH)[0]

{'pct(train)': 0.809998757918271, 'pct(dev)': 0.09000124208172898, 'pct(test)': 0.1}


In [11]:
texts = SentenceIterator(X_train)

with open('data.txt', 'w') as f:
  first = True
  for line in texts:
    if not first: f.write('\n')
    f.write(line)
    first = False

100%|██████████| 130426/130426 [00:21<00:00, 6127.08it/s]


In [10]:
# solo con un sample de la data
!head -n 100000 data.txt > hdata.txt

In [11]:
import fasttext
model = fasttext.train_unsupervised(
    'hdata.txt', model='skipgram',# epoch=10, minn=7, maxn=12
)


In [12]:
model.save_model(os.path.join(PATH, "full_fasttext_normalized.bin"))

In [14]:
import fasttext
model = fasttext.load_model(os.path.join(PATH, "full_fasttext_normalized.bin"))

In [ ]:
model.get_subwords('departamento')

In [9]:
from IPython.display import display_markdown
from random import choice

for i in range(30):
  word = choice(model.words)
  display_markdown(f'most similar to **{word}**', raw=True)
  for sim, word2 in model.get_nearest_neighbors(word):
    display_markdown(f'* {word2} ({sim:.02f})', raw=True)
  display_markdown(f'___')

most similar to **discordancias**

* inexactitudes (1.00)

* facturas (0.99)

* arrojar (0.98)

* títulos (0.97)

...

* casos (0.93)

* meses (0.93)

* sean (0.93)

* gestoría (0.93)

* art (0.93)

In [13]:
from IPython.display import display_markdown
from random import choice

words = ['cdra', 'elena', 'daniel', 'hs', 'señorial', 'vinílicos', 'by', 'monserrat', ]
for word in words:
  display_markdown(f'most similar to **{word}**', raw=True)
  for sim, word2 in model.get_nearest_neighbors(word):
    display_markdown(f'* {word2} ({sim:.02f})', raw=True)
  display_markdown(f'___')

most similar to **cdra**

* olazábal (0.82)

* cuadra (0.81)

* talcahuano (0.81)

* olazabal (0.81)

...

* dumont (0.83)

* chivilcoy (0.83)

* chacarita (0.82)

* boedo (0.82)

* pompeya (0.82)